In [ ]:
# add YOLO submodules as modules
import sys
import pathlib
sys.path.append(f"{pathlib.Path().absolute()}/YOLO")

# import yolo
from yolo import *

# change cwd to YOLO
import os
os.chdir('YOLO')

In [ ]:
# Model Initialization
YOLO_model = YOLO()

# Model Prediction returns classes for which number 2 stands for car
ENUM_CAR_CLASS = 2

# Model return rectangle coordinates, that represents area, where object is located
# We use this constant to determine if this area is small enough to be ignored
ERROR_DEVIATION = 0.2

# Car Detection

In [ ]:
def detect_car(path):
    # 1. Open Image
    image = Image.open(path)
    
    # 2. Preprocessing 
    image_data = preprocess_image(image, YOLO_model.model_image_size)

    # prepare origin image shape, (height, width) format
    image_shape = np.array([image.size[1], image.size[0]])
    image_shape = np.expand_dims(image_shape, 0)

    # 3. Prediction
    boxes, classes, scores = YOLO_model.predict(image_data, image_shape)
    
    # 4. Check if car is in the detected objects
    for i in range(len(classes)):
        if classes[i] == ENUM_CAR_CLASS:
            # 4.1. If car is detected check for box area
            box = boxes[i]
            area = abs((box[2] - box[0]) * (box[3] - box[1]))
            
            # 4.2. Normalize evaluated area
            area = area/(image_shape[0][0]*image_shape[0][1])

            # 4.3. If box area is too small, we can ignore it
            if area > ERROR_DEVIATION:
                return True
    return False